In [1]:
#!pip install tensorflow
#!pip install keras-tuner

In [120]:
import numpy as np
import tensorflow as tf
import pandas as pd
import os
os.environ['KERAS_BACKEND']='tf'
import keras
import keras_tuner 

In [121]:
print(keras.__version__)
print(tf.__version__)

2.13.1
2.13.0


In [122]:
from keras import layers
from kerastuner.tuners import RandomSearch
from keras.models import Sequential
from keras.layers import Dense
    


In [123]:
#read input csv - regression problem
df=pd.read_csv('carPrice_cleanedDf.csv')
df.head()

,car_ID,symboling,wheelbase,carlength,carwidth,curbweight,enginesize,boreratio,stroke,compressionratio,...,encoded_CarName,encoded_fueltype,encoded_aspiration,encoded_doornumber,encoded_carbody,encoded_drivewheel,encoded_enginelocation,encoded_enginetype,encoded_cylindernumber,encoded_fuelsystem
0,1,3,88.6,168.8,64.1,2548,130.0,3.47,2.68,9.0,...,13296.555064,13013.998834,12648.655088,13018.602727,16506.881607,19139.402392,12975.984777,15916.550259,10462.734353,17324.036276
1,2,3,88.6,168.8,64.1,2548,130.0,3.47,2.68,9.0,...,13569.736882,13013.998834,12648.655088,13018.602727,16506.881607,19139.402392,12975.984777,15916.550259,10462.734353,17324.036276
2,3,1,94.5,171.2,65.5,2823,152.0,2.68,3.47,9.0,...,13569.736882,13013.998834,12648.655088,13018.602727,10739.159659,19139.402392,12975.984777,19958.526335,20614.444286,17324.036276
3,4,2,99.8,176.6,66.2,2337,109.0,3.19,3.40,10.0,...,13337.918701,13013.998834,12648.655088,13483.196846,14243.557601,9549.877736,12975.984777,11681.811853,10462.734353,17324.036276
4,5,2,99.4,176.6,66.4,2824,136.0,3.19,3.40,8.0,...,13993.925476,13013.998834,12648.655088,13483.196846,14243.557601,12239.698564,12975.984777,11681.811853,17652.489177,17324.036276


In [124]:
X=df.drop(columns='price')

In [125]:
y=df.price

In [129]:
#model uilding using hyperparameters
def build_model(hp):
    model=Sequential()
    for i in range(hp.Int('num_layers',2,20)):
        print(i)
        print(hp.Int('num_layers',2,20))
        model.add(Dense(units=hp.Int('units_'+str(i),
                                     min_value=32,
                                     max_value=512,
                                     step=32),activation='relu'))
        print('units_'+str(i))
    model.add(Dense(1,activation='linear'))
    model.compile(keras.optimizers.Adam(hp.Choice('learning_rate',[0.01,0.001,0.0001])),
                      loss='mean_absolute_error',
                      metrics=['mean_absolute_error'])
        
    return model
                        
                        

In [131]:
#hyperparameter tuner
tuner=RandomSearch(build_model,
                   objective='val_mean_absolute_error',
                   max_trials=10,
                   max_retries_per_trial=3,
                   directory='keras_tuner'
                   )

Reloading Tuner from keras_tuner\untitled_project\tuner0.json


In [78]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [79]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=2024)


In [80]:
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.fit_transform(X_test)

In [81]:
y_train

108    13200.0
80      9959.0
25      6692.0
4      17450.0
115    16630.0
        ...   
36      7295.0
27      8558.0
128    37028.0
96      7499.0
136    18150.0
Name: price, Length: 143, dtype: float64

In [82]:
#reshaping the y_train for 1d to 2d as standard scaler require a 2d array
y_train=np.array(y_train).reshape(-1,1)
y_train=sc.fit_transform(y_train)

In [83]:
y_test=np.array(y_test).reshape(-1,1)
y_test=sc.fit_transform(y_test)

In [84]:
from keras.callbacks import EarlyStopping
#earlystopping is used when we don't know how many epochs needs to be used to optimize the model so early stopping will stop after sertain epochs when there is no
#imporvment in validation score
early_stopping_monitor=EarlyStopping(patience=2)



In [85]:
tuner.search(X_train,y_train,epochs=500,validation_data=(X_test,y_test),callbacks=[early_stopping_monitor])

Trial 10 Complete [00h 00m 22s]
val_mean_absolute_error: 0.26044273376464844

Best val_mean_absolute_error So Far: 0.1268094778060913
Total elapsed time: 00h 02m 54s


In [86]:
tuner.results_summary()


Results summary
Results in keras_tuner\untitled_project
Showing 10 best trials
Objective(name="val_mean_absolute_error", direction="min")

Trial 00 summary
Hyperparameters:
num_layers: 9
units_0: 352
units_1: 320
learning_rate: 0.001
units_2: 32
units_3: 32
units_4: 32
units_5: 32
units_6: 32
units_7: 32
units_8: 32
Score: 0.1268094778060913

Trial 02 summary
Hyperparameters:
num_layers: 8
units_0: 288
units_1: 224
learning_rate: 0.001
units_2: 128
units_3: 512
units_4: 480
units_5: 32
units_6: 416
units_7: 480
units_8: 352
Score: 0.13036967813968658

Trial 05 summary
Hyperparameters:
num_layers: 8
units_0: 160
units_1: 320
learning_rate: 0.001
units_2: 160
units_3: 288
units_4: 96
units_5: 32
units_6: 288
units_7: 480
units_8: 64
units_9: 512
units_10: 384
units_11: 448
units_12: 288
units_13: 384
units_14: 512
units_15: 448
units_16: 352
units_17: 256
units_18: 288
units_19: 480
Score: 0.1331872195005417

Trial 06 summary
Hyperparameters:
num_layers: 13
units_0: 192
units_1: 96
learn